In [2]:
import pandas as pd
import numpy as np
from numpy import diff

from scipy.stats import ttest_ind

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import ternary # ternary plot has a bug
import plotly.express as px
import h5py
import matplotlib as mpl

# import geopandas as gpd
import math

from matplotlib.patches import Circle, Wedge, Polygon

import os
import matplotlib.patheffects as path_effects


from netCDF4 import Dataset as NetCDFFile 

# for beachball - there are some error with using obspy
from pyrocko.plot import beachball
from pyrocko import moment_tensor as ptm

import cartopy.crs as ccrs
import cartopy.mpl.geoaxes

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from pathlib import Path
%matplotlib inline

import harmonica as hm

from obspy.core import UTCDateTime

from datetime import datetime, timezone

import seaborn as sns
sns.set()
sns.set_style(style='white') 
sns.set_style("ticks", {"xtick.major.size": 0.5, "ytick.major.size": 0.5})


#set the font globally
plt.rcParams.update({'font.family':'Helvetica'})

In [3]:
# data = pd.read_csv('sum_with_2005_eqs.csv', header = 'infer')
data = pd.read_csv("/Users/mohd0001/PhD_Monash/Paper2/time-series/data/SUeq_data.csv", header = 'infer')
# data.dtypes

# data = data[(data['MW'] >= 5.1)]

data.sort_values(by = 'MW', ascending = False, inplace = True)

data['DATE'] = pd.to_datetime(data['DATE']) # change dtype to datetime format
data['DATE'] = data['DATE'].dt.strftime('%Y-%m-%d') # change dtype to object

# data['DATE'] and data['TIME'] should be object (str) to perform + operator

data_dummy = np.array(data['DATE'] + ' ' + data['TIME'])

data_ev_time = []

for val in data_dummy:
    data_ev_time.append(UTCDateTime(val))

    
ref_date = data_ev_time[0]

data_ev_time = np.array(data_ev_time)

date = []

for val in data_ev_time:
    date.append(val - ref_date)
    
date = np.array(date) # this is in seconds

data['YEARS'] = date/31536000

data = data.sort_values(by ='YEARS', ascending=True) # this is an important step
# data = data[(data['DATE'] > '01-01-2003') & (data['DATE'] < '26-11-2004')]

In [4]:
## Deep Java
# data_dgoing = data.loc[data['CLASS'] == 'DGOING']
# data_dgoing = data_dgoing[(data_dgoing['LAT'] <= -2) & (data_dgoing['LON'] <= 120) &
#                           (data_dgoing['TYPE'] != 'THRUST') & # except thrust events
#                          (data_dgoing['DEPTH'] < 290) &
#                          (data_dgoing['MW'] >= 6 )]

# Intermediate depth
data_dgoing = data.loc[data['CLASS'] == 'DGOING']
data_dgoing = data_dgoing[(data_dgoing['LAT'] <= -2) & (data_dgoing['LON'] <= 120) &
                         (data_dgoing['DEPTH'] > 290) & (data_dgoing['DEPTH'] < 700) &
                         (data_dgoing['MW'] >= 6 )]

In [5]:
# # Sumatra-Andaman - short slab
# data_dgoing = data.loc[data['CLASS'] == 'DGOING']
# data_dgoing = data_dgoing[(data_dgoing['LAT'] > -2) &
#                          (data_dgoing['DEPTH'] > 70) &
#                          (data_dgoing['MW'] >= 6 )]

In [6]:
# olive band
t1 = -29
t2 = -28

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('Olive band')
print('Long slab')
# print('Max depth', data_dg.DEPTH.max())
# print('Max mag', data_dg.MW.max())
# print('Min mag', data_dg.MW.min())
print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 6.3) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]

print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

Olive band
Long slab
        LAT      LON  DEPTH      YEARS   MW        DATE   CLASS  TYPE
2466 -7.513  119.861  612.8 -28.945236  6.7  1976-01-23  DGOING  NORM
Megathrust
        LAT     LON  DEPTH      YEARS   MW        DATE    CLASS    TYPE
1428  3.438  96.246   22.2 -28.536944  7.0  1976-06-20  MTHRUST  THRUST


In [7]:
# green band
t1 = -26
t2 = -23

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('Green band')
print('Long slab')
# print('Max depth', data_dg.DEPTH.max())
# print('Max mag', data_dg.MW.max())
# print('Min mag', data_dg.MW.min())
print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 6.3) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]

print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

Green band
Long slab
       LAT      LON  DEPTH      YEARS   MW        DATE   CLASS  TYPE
324 -6.197  112.087  632.7 -23.391889  6.1  1981-08-11  DGOING  NORM
Megathrust
        LAT      LON  DEPTH      YEARS   MW        DATE    CLASS    TYPE
2170 -4.467  101.946   46.8 -25.137003  6.3  1979-11-13  MTHRUST  THRUST
96   -5.384  103.047   47.3 -24.394644  6.4  1980-08-10  MTHRUST  THRUST


In [8]:
# orange band
t1 = -21
t2 = -16

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('Orange band')
print('Long slab')
# print('Max depth', data_dg.DEPTH.max())
# print('Max mag', data_dg.MW.max())
# print('Min mag', data_dg.MW.min())
print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 6.3) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]

print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

Orange band
Long slab
       LAT      LON  DEPTH      YEARS   MW        DATE   CLASS  TYPE
398 -5.771  111.228  537.6 -20.313487  6.6  1984-09-07  DGOING   ODD
255 -6.086  113.434  589.4 -19.395793  6.2  1985-08-08  DGOING  NORM
Megathrust
        LAT     LON  DEPTH      YEARS   MW        DATE    CLASS    TYPE
1361  0.173  97.953   28.1 -20.120661  7.1  1984-11-17  MTHRUST  THRUST


In [9]:
# maroon band
t1 = -13
t2 = -12

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('Maroon band')
print('Long slab')
# print('Max depth', data_dg.DEPTH.max())
# print('Max mag', data_dg.MW.max())
# print('Min mag', data_dg.MW.min())
print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 6.3) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]

print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

Maroon band
Long slab
       LAT      LON  DEPTH      YEARS   MW        DATE   CLASS  TYPE
323 -6.065  112.068  624.2 -12.887688  6.6  1992-02-09  DGOING  NORM
Megathrust
      LAT      LON  DEPTH      YEARS   MW        DATE    CLASS    TYPE
95 -5.469  102.897   28.4 -12.697382  6.6  1992-04-18  MTHRUST  THRUST
98 -5.758  103.032   38.3 -12.574554  6.3  1992-06-02  MTHRUST  THRUST


In [10]:
# purple band
t1 = -11
t2 = -8

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('Purple band')
print('Long slab')
# print('Max depth', data_dg.DEPTH.max())
# print('Max mag', data_dg.MW.max())
# print('Min mag', data_dg.MW.min())
print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 6.3) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]

print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

Purple band
Long slab
       LAT      LON  DEPTH      YEARS   MW        DATE   CLASS  TYPE
374 -5.806  110.310  637.6 -10.252281  6.6  1994-09-28  DGOING  NORM
377 -5.801  110.377  639.7 -10.252224  6.0  1994-09-28  DGOING  NORM
375 -5.671  110.169  561.9 -10.119079  6.5  1994-11-15  DGOING  NORM
Megathrust
        LAT      LON  DEPTH      YEARS   MW        DATE    CLASS    TYPE
944  -2.003   99.715   20.2 -10.146476  6.4  1994-11-05  MTHRUST  THRUST
2056 -4.996  103.140   40.0  -9.635957  6.3  1995-05-11  MTHRUST  THRUST


In [11]:
# cyan band
t1 = -8
t2 = -6

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('Cyan band')
print('Long slab')
# print('Max depth', data_dg.DEPTH.max())
# print('Max mag', data_dg.MW.max())
# print('Min mag', data_dg.MW.min())
print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 6.3) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]

print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

Cyan band
Long slab
Empty DataFrame
Columns: [LAT, LON, DEPTH, YEARS, MW, DATE, CLASS, TYPE]
Index: []
Megathrust
        LAT     LON  DEPTH     YEARS   MW        DATE    CLASS    TYPE
1516 -0.565  99.191   34.4 -6.980827  6.9  1998-01-04  MTHRUST  THRUST


In [12]:
# red band
t1 = -5
t2 = -2

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('Red band')
print('Long slab')
# print('Max depth', data_dg.DEPTH.max())
# print('Max mag', data_dg.MW.max())
# print('Min mag', data_dg.MW.min())
print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 6.3) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]

print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

Red band
Long slab
        LAT      LON  DEPTH     YEARS   MW        DATE   CLASS    TYPE
2549 -7.167  117.443  521.9 -3.860273  6.0  2001-02-16  DGOING    NORM
131  -7.545  115.616  316.0 -2.800015  6.0  2002-03-10  DGOING  THRUST
Megathrust
        LAT      LON  DEPTH     YEARS   MW        DATE    CLASS    TYPE
566  -4.074  101.597   34.1 -3.945470  6.8  2001-01-16  MTHRUST  THRUST
2054 -4.837  102.365   34.0 -3.868836  7.3  2001-02-13  MTHRUST  THRUST
1758  2.837   96.078   28.9 -2.874320  7.2  2002-02-11  MTHRUST  THRUST
1760  2.930   96.380   27.0 -2.874091  6.3  2002-02-11  MTHRUST  THRUST


In [13]:
# yellow band
t1 = -1
t2 = 1

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('Yellow band')
print('Long slab')
# print('Max depth', data_dg.DEPTH.max())
# print('Max mag', data_dg.MW.max())
# print('Min mag', data_dg.MW.min())
print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 7) & # removed foreshocks and aftershocks
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]

print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

Yellow band
Long slab
        LAT      LON  DEPTH     YEARS   MW        DATE   CLASS  TYPE
2223 -2.532  103.932  582.2 -0.422066  7.3  2004-07-25  DGOING  NORM
Megathrust
        LAT     LON  DEPTH     YEARS   MW        DATE    CLASS    TYPE
1426  3.299  95.876   28.6  0.000000  9.0  2004-12-26  MTHRUST  THRUST
835   6.930  92.906   45.7  0.001815  7.2  2004-12-26  MTHRUST  THRUST
2625  3.026  97.459   25.8  0.252502  8.6  2005-03-28  MTHRUST     NaN


In [14]:
# grey band
t1 = 2
t2 = 3

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('Grey band')
print('Long slab')
# print('Max depth', data_dg.DEPTH.max())
# print('Max mag', data_dg.MW.max())
# print('Min mag', data_dg.MW.min())
print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 7) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]

print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

Grey band
Long slab
       LAT      LON  DEPTH     YEARS   MW        DATE   CLASS TYPE
424 -5.917  107.423  295.0  2.616422  7.5  2007-08-08  DGOING  ODD
Megathrust
        LAT      LON  DEPTH     YEARS   MW        DATE    CLASS    TYPE
939  -2.174   99.538   20.0  2.714922  7.0  2007-09-13  MTHRUST  THRUST
2169 -4.458  101.359   32.0  2.953983  8.5  2007-12-09  MTHRUST  THRUST


In [15]:
# No correlation after great meagthrust earthquakes

t1 = 3
t2 = 10

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1) & (data_dgoing['YEARS'] < t2)]

print('After 2007 EQ')
print('Long slab')

print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 7) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]


print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

After 2007 EQ
Long slab
        LAT      LON  DEPTH     YEARS   MW        DATE   CLASS    TYPE
2594 -6.917  116.753  517.4  6.772934  6.6  2011-10-03  DGOING    NORM
2468 -7.298  119.944  544.5  8.742238  6.1  2013-09-21  DGOING  O_NORM
Megathrust
        LAT      LON  DEPTH     YEARS   MW        DATE    CLASS    TYPE
1757  2.742   95.933   32.0  3.153756  7.3  2008-02-20  MTHRUST  THRUST
648  -2.491   99.889   30.9  3.166891  7.2  2008-02-25  MTHRUST  THRUST
1943  2.391   97.084   31.8  5.442221  7.8  2010-06-04  MTHRUST  THRUST
1427  3.750   96.028   41.4  5.695892  7.2  2010-09-05  MTHRUST  THRUST
487  -3.502  100.097   22.8  5.832788  7.8  2010-10-25  MTHRUST  THRUST


In [21]:
# Mw < 6 in the long slab after 2007


data_dgoing = data.loc[data['CLASS'] == 'DGOING']
data_dgoing = data_dgoing[(data_dgoing['LAT'] <= -2) & (data_dgoing['LON'] <= 120) &
                         (data_dgoing['DEPTH'] > 290) & (data_dgoing['DEPTH'] < 700) &
                         (data_dgoing['MW'] < 6 )]

t1 = 3
t2 = 10

data_dg = data_dgoing[(data_dgoing['YEARS'] > t1)]

print('After 2007 EQ')
print('Long slab')

print(data_dg[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])


data_mt = data[(data['LAT'] < 15) & (data['LAT'] > -6) &
        (data['MW'] >= 7) &
           (data['CLASS'] == 'MTHRUST') &
           (data['YEARS'] > t1) & (data['YEARS'] < t2)]


print('Megathrust')
# print('Max depth', data_mt.DEPTH.max())
# print('Max mag', data_mt.MW.max())
# print('Min mag', data_mt.MW.min())
print(data_mt[['LAT','LON','DEPTH', 'YEARS', 'MW', 'DATE', 'CLASS', 'TYPE']])

After 2007 EQ
Long slab
        LAT      LON  DEPTH     YEARS   MW        DATE   CLASS      TYPE
2599 -7.655  116.811  324.1  3.519615  5.7  2008-07-02  DGOING  O_THRUST
384  -5.642  110.067  544.2  3.876710  5.2  2008-11-10  DGOING      NORM
270  -6.105  112.872  593.6  5.703943  5.3  2010-09-08  DGOING      NORM
2597 -7.069  116.935  615.9  6.926926  5.8  2011-11-28  DGOING    O_NORM
2615 -7.700  117.120  301.3  8.643859  5.1  2013-08-16  DGOING    THRUST
2616 -7.667  117.044  299.1  9.443144  5.1  2014-06-03  DGOING    THRUST
2550 -7.008  117.494  542.3  9.849989  5.8  2014-10-30  DGOING    O_NORM
Megathrust
        LAT      LON  DEPTH     YEARS   MW        DATE    CLASS    TYPE
1757  2.742   95.933   32.0  3.153756  7.3  2008-02-20  MTHRUST  THRUST
648  -2.491   99.889   30.9  3.166891  7.2  2008-02-25  MTHRUST  THRUST
1943  2.391   97.084   31.8  5.442221  7.8  2010-06-04  MTHRUST  THRUST
1427  3.750   96.028   41.4  5.695892  7.2  2010-09-05  MTHRUST  THRUST
487  -3.502  100.097 